In [69]:
import requests
import json

# api_server_url = "http://localhost:41001"
api_server_url = "https://allsecure.co.kr/api"
api_server_url = "https://77c5-59-13-206-70.ngrok-free.app"

# 1. Controller API

### GET /list_models 

In [71]:
ret = requests.get(api_server_url + "/list_models")

output_json = ret.json()
print(output_json)

{'models': ['COKAL-ko-v1-70B-GPTQ', 'MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ', 'Mixtral-8x7B-Instruct-v0.1-GPTQ', 'MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ']}


### GET /refresh_all_models

In [21]:
ret = requests.get(api_server_url + "/refresh_all_models")

### GET /get_random_uuid

In [3]:
ret = requests.get(api_server_url + "/get_random_uuid")

output_json = ret.json()
print(output_json)

{'uuid': '126e8b00e48a453c820219e9c6179ca8'}


### (deprecated) {controller_address}/get_worker_address

model_name = output_json['models'][1]
print("model_name:", model_name)

input_json = {
    "model": model_name
}

ret = requests.post(
    controller_url + "/get_worker_address", 
    json=input_json
)

output_json = ret.json()
print(output_json)

# 2. Worker API

### POST /model_details 

In [104]:
input_json = {
    "model_name": "llama2-13b-dpo-v4",
}

ret = requests.post(
    api_server_url + "/model_details",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'context_length': 8192}


### POST /worker_get_conv_template  

In [5]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
}

ret = requests.post(
    api_server_url + "/worker_get_conv_template",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'conv': {'name': 'chat-orca', 'system_template': '{system_message}', 'system_message': '### System:\nThis is a system prompt, please behave and help the user.', 'roles': ['### User', '### Assistant'], 'messages': [], 'offset': 0, 'sep_style': 2, 'sep': '\n\n', 'sep2': '</s>', 'stop_str': None, 'stop_token_ids': None, 'config': {'max_new_tokens': 4096}, 'translator_system': None, 'retrieval_system': None}}


### POST /count_token 

In [111]:
input_json = {
    "model_name": "llama2-13b-dpo-v4",
    # "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: NFC는 무엇의 약자입니까?\n\n### Assistant:"
    "prompt": """물론이죠! 여기 25가지 여름철에 아이들이 즐길 수 있는 재미있고 건강한 활동 목록이 있습니다:""",
}

ret = requests.post(api_server_url + "/count_token", json=input_json)

output_json = ret.json()
print(output_json)

{'count': 7478, 'error_code': 0}


### POST /worker_generate

In [ ]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ2",
    "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: NFC는 무엇의 약자입니까?\n\n### Assistant:",
    "temperature": 0.7,
    "top_p": 0.8,
    "session_id": 3,
}

ret = requests.post(
    api_server_url + "/worker_generate",
    json=input_json,
)

output_json = ret.json()
print(output_json)

### POST /worker_generate_stream

In [78]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
    "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: react hello world\n\n### Assistant: 반응 프로젝트에서 \"Hello World\"를 표시하려면 다음 단계를 따르십시오.\n1. 새로운 React 응용 프로그램 생성 :\n먼저 새로운 React 응용 프로그램을 만들어야합니다.\n 다음 명령을 실행하여 새로운 React 앱을 만듭니다.\n```\nNPX Create-React-App My-App\n```\n이 명령은 새로운 폴더`My-App '에서 새로운 React 프로젝트를 만듭니다.\n2. 구성 요소 만들기 :\n이제 앱의 루트 구성 요소를 만들어야합니다.\n 터미널에서 다음 명령을 실행하여 새 구성 요소를 만듭니다.\n```\nCD My-App\nNPX React-S 생성 구성 요소 환경 적 응용 프로그램\n```\n이 명령은`src` 디렉토리에`myapp.js`라는 새 파���일을 생성합니다.\n3. 구성 요소 업데이트 :\n`myapp.js` 파일을 열고 다음 코드를 추가하십시오.\n```JS\n'React'에서 React React React;\n함수 myapp () {\n   반환 (\n     <h1> 안녕하세요, 세계! </h1>\n   );\n}\n수출 기본 MyApp;\n```\n이 코드는 \"Hello, World!\"텍스트를 표시하는 간단한 구성 요소를 만듭니다.\n4. 구성 요소 렌더링 :\n이제 앱의 기본 구성 요소를 렌더링해야합니다.\n `src/index.js` 파일을 열고 코드를 업데이트하십시오.\n```JS\n'React'에서 React React React;\n'./myapp'에서 myapp 가져 오기;\n기능 실행 () {\n   반환 (\n     <MyApp />\n   );\n}\n기본 기능 내보내기;\n```\n5. 앱 실행 :\n마지막으로 앱을 실행해보세요.\n```\nNPX 시작\n```\n이렇게하면 웹 브라우저에서 응용 프로그램을 실행하는 데 사용할 수있는 개발 서버가 시작됩니다.\n 응용 프로그램을 열고 브라우저에서 \"Hello, World!\"텍스트를 표시해야합니다.\n번역 번역\n반응 반응 프로젝트 프로젝트에서에서 \"hello world\"를 표시하려면하려면 다음",
    "max_new_tokens": 1024,
    "temperature": 0.7,
    "top_p": 0.8,
    "session_id": 3,
    "repetition_penalty": 1.7,
}

ret = requests.post(
    api_server_url + "/worker_generate_stream",
    json=input_json,
    stream=True,
    # timeout=200,
)

past_len = len(input_json["prompt"])
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

 다음 단계를 따르십시오 따르십시오.
1. 1. 새 새 새 응용 프로그램 프로젝트 :
새 새 새 응용 프로그램 프

KeyboardInterrupt: 

In [ ]:
ret.text

In [ ]:
ret.text

### POST /worker_generate_auto

In [53]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "user_input": "NFC는 무엇의 약자입니까?",
    "session_id": 0,
    "temperature": 0.7,
    "top_p": 0.8,
}

ret = requests.post(
    api_server_url + "/worker_generate_auto",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'text': ' NFC는 나노 필름 코퍼레이션의 약자입니다.'}


### POST /worker_generate_stream_auto

In [70]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "user_input": "주식회사 아도스의 직원 수는 몇명인가요?",
    "session_id": 0,
    "context": '\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프트웨어 개발,게임 개발\n- 주소\n-\n서울 송파구 송파대로 167, B동 1021호지도보기\n- SNS\n-\n- 보유기술\n-\n-\n특허권\n- 디스플레이 해킹 방지 시스템 및 그 방법(2020)\n-\n특허권\n- 웹 클라이언트를 이용한 망 분리 시스템 및 그 방법(2020)\n-\n특허권\n- 컴퓨터의 시스템자원 및 프로세스의 보호 및 격리장치와 그방법(2020)\n-\n특허권\n- PKI 기반의 사물인터넷 기기 인증방법 및 인증시스템(2020)\n- 특허권\n미래 혹은 가까에 있는 위협으로 부터, 우리의 생명,\n\n\n(주)아도니스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도니스 기업소개 - 업력 : 35년차, 기업형태 : -, 업종 : 골프장 운영업 | (주)아도니스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 궁금하시다면, ...- 사업내용\n-\n골프장,호텔 운영/무역,담배,골프용품 도소매,통신판매/한식,중식 음식점\n- 주소\n-\n경기 포천시 신북면 포천로 2499지도보기\n기업비전\n주요 사업내용 : 포천 아도니스CC 운영\n주요 사업내용 : 포천 아도니스CC 운영\n주요 취급품목 : 골프장운영, 호텔운영\n우리는 단단하게 성장하고 있어요\n-\n2005\n-\n01월\n호텔 준공 영업개시\n- 01월\n-\n1999\n-\n06월\n상호변경 : (주)대우레져 -> (주)아도니스\n-\n04월\n골프장 개장\n- 06월\n-\n1995\n-\n04월\n골프장 착공허가 (경기도 포천군)\n- 04월\n-\n1\n990\n-\n10월\n경기도로부터 골프장사업계획을 승인\n- 10월\n-\n1989\n-\n02월\n회사 설립\n- 02월\n인원 규모를 파악해 보세요!\n전체 직원 수 56명\n- 전체인원\n- 입사자\n- 퇴사자\n',
    "max_new_tokens": 56,
    "temperature": 0.7,
    "top_p": 0.8,
}

ret = requests.post(
    api_server_url + "/worker_generate_stream_auto",
    json=input_json,
    stream=True,
    # timeout=2,
)

# past_len = len(input_json["prompt"])
past_len = 0
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

  주식회사 아도스의 직원 수는 56명입니다.

### POST /worker_get_last_prompt

In [11]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_get_last_prompt",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'prompt': '### System:\nThis is a system prompt, please behave and help the user.\nDate:2023-11-13 15:21\nYour reply should be based on the context below.\n\nContext:\n(주)아도스 2023년 기업정보 | 사원수, 회사소개, 근무환경 ...\n(주)아도스 기업소개 - 업력 : 10년차, 기업형태 : 중소기업, 업종 : 응용 소프트웨어 개발 및 공급업 | (주)아도스의 사원수, 연봉, 채용, 복리후생, 재무정보 등이 ... 재산, 가치를 구하는 것을 사명으로, 좋은 사람들이 모여 즐겁게 성장하고 있는 회사입니다.\n1. 현재는 IT보안이 주 사업 영역이며, AI와 가상화기술 기반의 정보보호 솔루션을 개발/운영하고 있습니다.\n2. 대기업 제조사의 네트워크 보안, 생산라인 보안, 정보 유출 방지 등 통합보안 서비스(모의 해킹, 진단, 관제, 이상징후 분석)를 담당하고 있습니다.\n"인류의 생명과 가치를 구한다"\n우리는 단단하게 성장하고 있어요\n-\n2021\n-\n09월\n가드랜드(정보보호솔루션) 출시\n-\n03월\n가두리(웹격리솔루션) 출시\n- 09월\n-\n2020\n\n-\n09월\n병역지정업체(산업체) 선정\n-\n05월\n벤처기업확인(기술보증기금,유효기간:2022.05.28)\n-\n04월\n수차에 걸쳐 자본금을 1200백만원으로 증자\n- 09월\n-\n2019\n-\n12월\n기업부설연구소 설립\n- 12월\n-\n2017\n-\n10월\n상호를 (주)아도스로 변경\n- 10월\n-\n2014\n-\n09월\n컴퓨터 프로그래밍 서비스업을 목적으로 대표이사 이승원에 의해 (주)피그플라이 설립\n- 09월\n우리는 직원의 복리후생도 중요해요\n-\n지원금/보험\n- 각종 경조사 지원\n-\n급여제도\n- 퇴직연금\n- 인센티브제\n- 스톡옵션\n- 퇴직금\n-\n업\n- 대표자명\n- 이승원\n- 사업내용\n-\n소프

### POST /session_history_clear

In [12]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/session_history_clear",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 0}


### POST /worker_regenerate_auto

In [13]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_auto",
    json=input_json,
)

output_json = ret.json()
print(output_json)

{'err_code': 70003}


### POST /worker_regenerate_stream_auto

In [14]:
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": 0,
}

ret = requests.post(
    api_server_url + "/worker_regenerate_stream_auto",
    json=input_json,
    stream=True,
)

# past_len = len(input_json["prompt"])
past_len = 0
for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
    if chunk:
        data = json.loads(chunk.decode())
        if "text" not in data:
            continue
        print(data["text"][past_len:], sep="", end="")
        past_len = len(data["text"])

### POST /worker_stop_stream

In [15]:
import uuid
from multiprocessing import Process
import time


# background runner for worker_generate_stream
def background_run_stream(session_id):
    input_json = {
        "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
        "prompt": "### System:\nThis is a system prompt, please behave and help the user.\n\n### User: 유튜브에 대해서 설명해주세요.\n\n### Assistant:",
        "session_id": session_id,
        "temperature": 0.7,
        "top_p": 0.8,
    }

    ret = requests.post(
        api_server_url + "/worker_generate_stream",
        json=input_json,
        headers={"User-Agent": "FastChat Client"},
        stream=True,
        timeout=200,
    )

    past_len = len(input_json["prompt"])
    for idx, chunk in enumerate(ret.iter_lines(decode_unicode=False, delimiter=b"\0")):
        if chunk:
            data = json.loads(chunk.decode())
            print(data["text"][past_len:], end="")
            past_len = len(data["text"])


# generate random session_id
session_id = uuid.uuid4().hex
print("session_id:", session_id)

# run worker_generate_stream
p1 = Process(target=background_run_stream, args=[session_id])
p1.start()

# wait for 3 seconds
time.sleep(3)

# send worker_stop_stream
input_json = {
    "model_name": "MingAI-70B-chat-orca_v0.4_v0.2_retrained_checkpoint-2-GPTQ",
    "session_id": session_id,
}

ret = requests.post(
    api_server_url + "/worker_stop_stream",
    json=input_json,
)

print("\n\n Generation Stopped")

session_id: b5bdc37a9ed14ac1b3d05e7ff5566659
 유튜브는 세계에서 가장 큰 온라인 동영상 공유 및 소셜 미디어 플

 Generation Stopped
